In [1]:
from secScraper import *
import sys

if sys.version_info[0] < 3 or sys.version_info[1] < 6:
    raise Exception("Must be using Python >= 3.6 due to reliance on ordered default dict.")
else:
    version = "[INFO] Running python {}.{}.{}".format(*sys.version_info[:3])
    if display.run_from_ipython():
        %load_ext autoreload
        %autoreload 2
        %matplotlib notebook
        version += " for ipython" if display.run_from_ipython() else ""
    print("[INFO] Running python {}.{}.{} (>= python 3.6)".format(*sys.version_info[:3]))

[INFO] Running python 3.7.3 (>= python 3.6)


In [2]:
import csv
import psycopg2
from tqdm import tqdm

# Read the settings in postgres

In [3]:
connector = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="1")

In [4]:
s = postgres.retrieve_settings(connector)

[WARNING] Verify that the following are meant to stay str:
path_stage_1_data : /home/alex/Desktop/filtered_text_data/nd_data/
path_stock_database : /home/alex/Desktop/Insight project/Database/Ticker_stock_price.csv
path_filtered_stock_data : /home/alex/Desktop/Insight project/Database/filtered_stock_data.csv
path_stock_indexes : /home/alex/Desktop/Insight project/Database/Indexes/
path_filtered_index_data : /home/alex/Desktop/Insight project/Database/Indexes/filtered_index_data.csv
path_lookup : /home/alex/Desktop/Insight project/Database/lookup.csv
path_filtered_lookup : /home/alex/Desktop/Insight project/Database/filtered_lookup.csv
path_master_dictionary : /home/alex/Desktop/Insight project/Database/LoughranMcDonald_MasterDictionary_2018.csv
path_dump_crsp : /home/alex/Desktop/Insight project/Database/dump_crsp_merged.txt
path_output_folder : /home/alex/Desktop/Insight project/Outputs
path_dump_cik_scores : /home/alex/Desktop/Insight project/Outputs/dump_cik_scores.csv
path_dump_pf_

# Dump all the stock data to postgres

## Load the time_range of the stock data of interest
The main CSV file will be read and any data point outside s['time_range'] will be ignored.

In [5]:
# Load all stock prices - takes a while but one off
stock_data = pre_processing.load_stock_data(s, penny_limit=75000000)

  0%|          | 72975/30563446 [00:00<01:46, 287207.22it/s]

[INFO] Loading data from (2012, 1) to (2018, 4)
[INFO] Starting the mapping


100%|██████████| 30563446/30563446 [01:39<00:00, 306984.79it/s]


[INFO] stock_data load statistics:
Incomplete lines: 49,490/30,563,446
Penny stocks found (at least one entry below threshold): 1905/5503
Lines out of range: 23,596,984/30,563,446


## Dump to CSV first
Why? Because somehow dumping to CSV then importing in postgres from CSV is crazy fast.

In [6]:
header_stock_data = (('TICKER', 'text'), ('DATE', 'date'),
                    ('ASK', 'float'), ('MARKET_CAP', 'bigint'))
with open(s['path_filtered_stock_data'], 'w') as f:
    out = csv.writer(f, delimiter=';')
    header = ['IDX'] + [name[0] for name in header_stock_data]
    out.writerow(header)
    idx = 0
    for ticker in tqdm(stock_data):
        for ts in stock_data[ticker]:
            #print(ts)
            out.writerow((idx, ticker, ts, stock_data[ticker][ts][0], int(stock_data[ticker][ts][1])))
            idx += 1

100%|██████████| 3598/3598 [00:10<00:00, 330.13it/s]


## copy_from csv to postgres

In [7]:
path = s['path_filtered_stock_data']
postgres.csv_to_postgres(connector, 'stock_data', header_stock_data, path)

[INFO] Deleted table stock_data
[INFO] Creating the following table:
CREATE TABLE stock_data(IDX integer PRIMARY KEY,TICKER text,DATE date,ASK float,MARKET_CAP bigint)
CPU times: user 435 ms, sys: 49.2 ms, total: 485 ms
Wall time: 10.5 s


# Optional: retrieve the data

In [8]:
sd = postgres.retrieve_all_stock_data(connector, 'stock_data')

SELECT * FROM stock_data;


100%|██████████| 4769013/4769013 [00:05<00:00, 930958.05it/s] 


In [9]:
sd.keys()

dict_keys(['AEPI', 'JJSF', 'PLXS', 'HGR', 'HNGR', 'ORCL', 'MSFT', 'TROW', 'HON', 'EMC', 'AMRC', 'XRIT', 'AKRX', 'TECD', 'RGEN', 'AT', 'BWC', 'BWXT', 'BEAM', 'BWINB', 'PTVCB', 'INDB', 'TKLC', 'HGIC', 'LSBG', 'LLTC', 'CY', 'TMP', 'BCPC', 'SXCL', 'DJCO', 'AMAG', 'TCB', 'TCF', 'ASTE', 'NAVG', 'WERN', 'SKYW', 'PATR', 'FRPH', 'AMWD', 'ADM', 'AAN', 'VIVO', 'WTS', 'FNB', 'PTSI', 'BRLI', 'HLS', 'EHC', 'FISV', 'FCNCA', 'HTLD', 'RENT', 'OSUR', 'BWS', 'CAL', 'BC', 'UIS', 'WSFS', 'ITIC', 'CERN', 'WBS', 'MKL', 'CMVT', 'CDR', 'CHKM', 'ACMP', 'WPZ', 'FLIC', 'FBP', 'STSA', 'DELL', 'SYNO', 'PARL', 'AVD', 'PLAB', 'CRMT', 'AGII', 'ARGO', 'OFG', 'CFNB', 'CATO', 'FBNC', 'WSBC', 'KO', 'SAFM', 'CNBKA', 'UBSI', 'ENV', 'CCC', 'CMRG', 'SKS', 'WASH', 'CDNS', 'ED', 'CHCO', 'BRY', 'EBIX', 'CECE', 'FICO', 'CNMD', 'CELG', 'HBNC', 'ATU', 'ATRI', 'ACO', 'XRAY', 'FAST', 'CPF', 'HRTX', 'LABL', 'AIRM', 'GDOT', 'DTE', 'ALEX', 'MATX', 'IMKTA', 'DD', 'CBM', 'AIN', 'IIVI', 'ETN', 'JOUT', 'SIVB', 'ALCO', 'CLH', 'CTBI', 'WPPGY'

# Dump all the index data to postgres
Essentially the same with the index_data

In [10]:
index_data = pre_processing.load_index_data(s)
print("[INFO] Loaded the following index data:", list(index_data.keys()))

[INFO] Loaded the following index data: ['IXIC', 'SPX', 'DJI', 'RUT']


In [11]:
index_data['IXIC']

{datetime.date(1993, 1, 4): 675.309998,
 datetime.date(1993, 1, 5): 670.52002,
 datetime.date(1993, 1, 6): 678.440002,
 datetime.date(1993, 1, 7): 681.559998,
 datetime.date(1993, 1, 8): 677.190002,
 datetime.date(1993, 1, 11): 677.460022,
 datetime.date(1993, 1, 12): 681.440002,
 datetime.date(1993, 1, 13): 679.210022,
 datetime.date(1993, 1, 14): 690.320007,
 datetime.date(1993, 1, 15): 692.460022,
 datetime.date(1993, 1, 18): 694.690002,
 datetime.date(1993, 1, 19): 698.380005,
 datetime.date(1993, 1, 20): 696.820007,
 datetime.date(1993, 1, 21): 696.47998,
 datetime.date(1993, 1, 22): 700.590027,
 datetime.date(1993, 1, 25): 703.659973,
 datetime.date(1993, 1, 26): 709.880005,
 datetime.date(1993, 1, 27): 705.820007,
 datetime.date(1993, 1, 28): 695.390015,
 datetime.date(1993, 1, 29): 696.960022,
 datetime.date(1993, 2, 1): 697.97998,
 datetime.date(1993, 2, 2): 702.25,
 datetime.date(1993, 2, 3): 707.679993,
 datetime.date(1993, 2, 4): 710.030029,
 datetime.date(1993, 2, 5): 706.

In [12]:
header_index_data = (('INDEX', 'text'), ('DATE', 'date'), ('ASK', 'float'))
with open(s['path_filtered_index_data'], 'w') as f:
    out = csv.writer(f, delimiter=';')
    header = ['IDX'] + [name[0] for name in header_index_data]
    out.writerow(header)
    idx = 0
    for ticker in tqdm(index_data):
        for ts in index_data[ticker]:
            #print(ts)
            out.writerow((idx, ticker, ts, index_data[ticker][ts]))
            idx += 1

100%|██████████| 4/4 [00:00<00:00, 59.03it/s]


In [13]:
path = s['path_filtered_index_data']
%time postgres.csv_to_postgres(connector, 'index_data', header_index_data, path)

[INFO] Deleted table index_data
[INFO] Creating the following table:
CREATE TABLE index_data(IDX integer PRIMARY KEY,INDEX text,DATE date,ASK float)
CPU times: user 5.13 ms, sys: 25 µs, total: 5.15 ms
Wall time: 57.9 ms


In [14]:
ind = postgres.retrieve_all_stock_data(connector, 'index_data')

100%|██████████| 26373/26373 [00:00<00:00, 1535421.61it/s]

SELECT * FROM index_data;


In [15]:
ind.keys()

dict_keys(['IXIC', 'SPX', 'DJI', 'RUT'])

# Dump the lookup table to postgres
Another database that is unfrequently changing, so we load it in postgres

In [16]:
lookup = pre_processing.load_lookup(s)
print("[INFO] Loaded {:,} CIK/Tickers correspondances.".format(len(lookup)))

[INFO] Loaded 13,738 CIK/Tickers correspondances.


In [17]:
header_lookup = (('CIK', 'integer'), ('TICKER', 'text'))
with open(s['path_filtered_lookup'], 'w') as f:
    out = csv.writer(f, delimiter=';')
    header = ['IDX'] + [name[0] for name in header_lookup]
    out.writerow(header)
    for idx, item in enumerate(tqdm(lookup.items())):
        out.writerow((idx, *item))

100%|██████████| 13738/13738 [00:00<00:00, 816270.46it/s]


In [18]:
path = s['path_filtered_lookup']
%time postgres.csv_to_postgres(connector, 'lookup', header_lookup, path)

[INFO] Deleted table lookup
[INFO] Creating the following table:
CREATE TABLE lookup(IDX integer PRIMARY KEY,CIK integer,TICKER text)
CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 26.7 ms


In [19]:
lookup, reverse_lookup = postgres.retrieve_lookup(connector)

SELECT * FROM lookup;


# Dump LM dictionary into postgres
Same, the dictionary is updated infrequently so it should go into postgres. But that will be for later.

TO DO: Import in postgres